# Analyse d'Impact Écologique

Ce notebook fonctionne à la fois localement et sur Google Colab.

In [ ]:
def setup_environment():
    import sys
    from pathlib import Path
    global project_root
    
    # Détecter si on est sur Colab
    try:
        import google.colab
        print("Environnement Colab détecté, configuration...")
        !git clone https://github.com/fabricehong/eco-certificates.git
        %cd eco-certificates
        project_root = Path().absolute()
    except ImportError:
        print("Environnement local détecté, configuration...")
        # En local, on ajoute juste le répertoire parent au PYTHONPATH
        project_root = Path().absolute().parent
        if str(project_root) not in sys.path:
            sys.path.append(str(project_root))

setup_environment()

In [ ]:
from pathlib import Path

from src.services.csv_parser import CsvParser
from src.services.score_calculator import ScoreCalculator
from src.services.certificate_generator import CertificateGenerator, ElementPosition

# Créer le parser et le calculateur
parser = CsvParser(project_root / "input/product and service form.csv", score_component_size=5)
calculator = ScoreCalculator(component_size=5)

# Configurer le générateur de certificats
x = 150
generator = CertificateGenerator(
    certificate_template=Path("images/certificate.png"),
    active_leaf=Path("images/active-leave.png"),
    inactive_leaf=Path("images/unactive-leave.png"),
    # Positions des scores
    local_position=ElementPosition(x=x, y=820),
    eco_position=ElementPosition(x=x, y=1100),
    living_position=ElementPosition(x=x, y=1350),
    # Positions des descriptions
    local_description_position=ElementPosition(x=x, y=740),
    eco_description_position=ElementPosition(x=x, y=1000),
    living_description_position=ElementPosition(x=x, y=1270),
    # Configuration des feuilles
    leaf_spacing=100,  # Espacement entre les feuilles
    leaf_width=90,     # Largeur des feuilles, la hauteur sera calculée pour garder le ratio
    # Configuration des labels
    label_position=ElementPosition(x=350, y=1700),
    font_path=Path("fonts/Arial.ttf"),
    bold_font_path=Path("fonts/Arial Bold.ttf"),
    font_size=60,  # Taille pour les labels
    description_font_size=50  # Taille pour les descriptions
)

# Créer le dossier output s'il n'existe pas
output_dir = project_root / "output"
output_dir.mkdir(exist_ok=True)

In [ ]:
# Lire et traiter les produits
csv_products = parser.parse_products()
products_count = 0

print("Produits parsés:")
for product in csv_products:
    print(product)

# Pour chaque produit
for csv_product in csv_products:
    # Calculer les scores
    product = calculator.transform_product(csv_product)
    print(f"Produit traité avec succès : {product.name}")
    products_count += 1
    
    # Générer le certificat avec le score approprié (product ou service)
    score = (product.product_evaluation if product.product_evaluation.values_found 
            else product.service_evaluation)
    
    # Générer le nom du fichier
    filename = product.name.lower().replace(" ", "_") + "_certificate.png"
    output_path = output_dir / filename
    
    # Générer le certificat
    generator.generate_certificate(score, output_path)

# Afficher les détails
print("\nDétails des produits avec scores :")
print("=" * 50)
for csv_product in csv_products:
    product = calculator.transform_product(csv_product)
    print(product)
    filename = product.name.lower().replace(" ", "_") + "_certificate.png"
    print(f"Certificat généré : output/{filename}")
    print("=" * 50)

print(f"\nNombre de produits traités : {products_count}")

In [ ]:
# Zipper les certificats générés
import shutil
import os
from pathlib import Path
from datetime import datetime

# Obtenir le chemin de la racine du projet
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

# Créer un nom de fichier zip avec la date
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"certificates_{timestamp}.zip"

# Définir le chemin de sortie selon l'environnement
IN_COLAB = False
try:
    from google.colab import files
    IN_COLAB = True
    output_dir = Path("/content")
except ImportError:
    output_dir = project_root

zip_path = output_dir / zip_filename
output_folder = project_root / "output"

# Zipper le dossier output
shutil.make_archive(str(zip_path.with_suffix("")), 'zip', str(output_folder))

# Gérer le téléchargement selon l'environnement
if IN_COLAB:
    # Dans Colab, déclencher le téléchargement
    files.download(str(zip_path))
    print(f"Les certificats ont été zippés dans {zip_filename}")
    print("Le téléchargement devrait démarrer automatiquement...")
else:
    # En local, afficher l'emplacement du fichier
    print(f"Les certificats ont été zippés dans : {zip_path}")